In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import re
import numpy as np

In [8]:
path = r"C:\Users\jakub\Downloads\chromedriver-win64\chromedriver.exe"
service = Service(path)
driver = webdriver.Chrome(service=service)
driver.get("https://arslege.pl/kodeks-karny/k1/s193/")
time.sleep(5)
div_elements = driver.find_elements(By.CLASS_NAME, "art_tresc")
div_texts = [div.text for div in div_elements]
driver.quit()

In [70]:
# split by regex
parts = [re.split(r'\npodlega |, podlega |\n§ |\n- podlega ', text) for text in div_texts]

In [71]:
# delete uchylony
for part in parts:
    for i in range(len(part)-1, -1, -1):
        if '(uchylony)' in part[i]:
            part.remove(part[i])

In [72]:
# podział na artykuły i kary
penalties = []
articles = []
for part in parts:
    for item in part:
        if 'karze' in item:
            penalties.append(item)
        elif (item.endswith('.') == False) and (item != ''):
            articles.append(item)

In [73]:
for i in range(len(penalties) - 1, -1, -1):
    if ('Tej samej karze' in penalties[i]) or ('Nie podlega karze' in penalties[i]) or ('Jeżeli' in penalties[i]):
        penalties.remove(penalties[i])

In [74]:
len(penalties)

410

In [75]:
len(articles)

410

In [76]:
# fill penalties to articles' size
articles.extend([None] * (len(penalties)-len(articles)))
dc = {"crime": articles, "penalty": penalties}
df = pd.DataFrame(dc)

In [77]:
df

,crime,penalty
0,§ 1. Kto wszczyna lub prowadzi wojnę napastniczą,karze pozbawienia wolności na czas nie krótszy...
1,3. Kto publicznie nawołuje do wszczęcia wojny ...,karze pozbawienia wolności od 3 miesięcy do la...
2,"§ 1. Kto, w celu wyniszczenia w całości albo w...",karze pozbawienia wolności na czas nie krótszy...
3,"2. Kto, w celu określonym w § 1, stwarza dla o...",karze pozbawienia wolności od lat 5 do 25.
4,"§ 1. Kto, biorąc udział w masowym zamachu lub ...",karze pozbawienia wolności na czas nie krótszy...
...,...,...
405,"Kto, w dokumentacji związanej z obrotem papier...",karze pozbawienia wolności do lat 3.
406,Kto puszcza w obieg podrobiony albo przerobion...,"grzywnie, karze ograniczenia wolności albo poz..."
407,"§ 1. Kto, w celu użycia lub puszczenia w obieg...",karze pozbawienia wolności do lat 3.
408,"Kto, w celu użycia w obrocie gospodarczym, pod...","grzywnie, karze ograniczenia wolności albo poz..."


In [79]:
# export to csv
df.to_csv('criminal.csv', sep=';', encoding='utf-8-sig')

In [63]:
# df.to_csv('test.csv', sep = ';', encoding='utf-8-sig')